<a href="https://colab.research.google.com/github/Fsrkh/Persian-dates-NLP/blob/main/date_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
!pip install tensorflow_text

In [46]:
import collections
import os
import pathlib
import re
import string
import sys
import tempfile
import time

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import tensorflow as tf
import tensorflow_text as tf_text

In [47]:
data = pd.read_csv("dates.csv")
data.head()

,Formal Date,Informal Date 1,Informal Date 2,Informal Date 3,Informal Date 4,Informal Date 5,Informal Date 6,Informal Date 7,Informal Date 8,Informal Date 9
0,1400/01/01,اول فروردین هزار و چهارصد,یک فروردین هزار و چهارصد,یک فروردین ۱۴۰۰,فروردین یک ۱۴۰۰,اول فروردین ۱۴۰۰,یکِ فروردین ۱۴۰۰,یکم فروردین ۱۴۰۰,اول فروردین ماه ۱۴۰۰,فروردینِ یک ۱۴۰۰
1,1400/02/10,دهم اردیبهشت هزار و چهارصد,ده اردیبهشت هزار و چهارصد,ده اردیبهشت ۱۴۰۰,اردیبهشت ده ۱۴۰۰,دهم اردیبهشت ۱۴۰۰,دهِ اردیبهشت ۱۴۰۰,دهم اردیبهشت ماه ۱۴۰۰,اردیبهشتِ ده ۱۴۰۰,ده اردیبهشت ۱۴۰۰
2,1401/03/05,پنجم خرداد هزار و چهارصد و یک,پنج خرداد هزار و چهارصد و یک,پنج خرداد ۱۴۰۱,خرداد پنج ۱۴۰۱,پنجم خرداد ۱۴۰۱,پنجِ خرداد ۱۴۰۱,پنجم خرداد ماه ۱۴۰۱,خردادِ پنج ۱۴۰۱,پنج خرداد ۱۴۰۱
3,1401/04/15,پانزدهم تیر هزار و چهارصد و یک,پانزده تیر هزار و چهارصد و یک,پانزده تیر ۱۴۰۱,تیر پانزده ۱۴۰۱,پانزدهم تیر ۱۴۰۱,پانزدهِ تیر ۱۴۰۱,پانزدهم تیر ماه ۱۴۰۱,تیرِ پانزده ۱۴۰۱,پانزده تیر ۱۴۰۱
4,1402/05/20,بیستم مرداد هزار و چهارصد و دو,بیست مرداد هزار و چهارصد و دو,بیست مرداد ۱۴۰۲,مرداد بیست ۱۴۰۲,بیستم مرداد ۱۴۰۲,بیستِ مرداد ۱۴۰۲,بیستم مرداد ماه ۱۴۰۲,مردادِ بیست ۱۴۰۲,بیست مرداد ۱۴۰۲


In [48]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 761 entries, 0 to 760
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Formal Date      761 non-null    object
 1   Informal Date 1  761 non-null    object
 2   Informal Date 2  761 non-null    object
 3   Informal Date 3  761 non-null    object
 4   Informal Date 4  761 non-null    object
 5   Informal Date 5  761 non-null    object
 6   Informal Date 6  138 non-null    object
 7   Informal Date 7  138 non-null    object
 8   Informal Date 8  138 non-null    object
 9   Informal Date 9  138 non-null    object
dtypes: object(10)
memory usage: 59.6+ KB


In [49]:
context_raw = data.iloc[:, 1:].astype(str).apply(lambda row: ' '.join(row.values), axis=1).to_numpy()
target_raw = data.iloc[:, 0].astype(str).to_numpy()

In [50]:
BUFFER_SIZE = len(context_raw)
BATCH_SIZE = 64

is_train = np.random.uniform(size=(len(target_raw),)) < 0.8

train_raw = (
    tf.data.Dataset
    .from_tensor_slices((context_raw[is_train], target_raw[is_train]))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE)
)

val_raw = (
    tf.data.Dataset
    .from_tensor_slices((context_raw[~is_train], target_raw[~is_train]))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE)
)

In [51]:
VOCAB_SIZE = 2000
MAX_LEN = 50

context_vectorizer = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE, output_mode='int', output_sequence_length=MAX_LEN)
target_vectorizer = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE, output_mode='int', output_sequence_length=MAX_LEN)

In [52]:
context_vectorizer.adapt(context_raw)
target_vectorizer.adapt(target_raw)

In [62]:
EMBEDDING_DIM = 64
UNITS = 128
DROPOUT_RATE = 0.3

encoder_input = tf.keras.Input(shape=(None,), dtype='int64', name='encoder_input')
encoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM)(encoder_input)
encoder_dropout = tf.keras.layers.Dropout(DROPOUT_RATE)(encoder_embedding)
encoder_lstm = tf.keras.layers.LSTM(UNITS, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_dropout)
encoder_states = [state_h, state_c]

In [63]:
decoder_input = tf.keras.Input(shape=(None,), dtype='int64', name='decoder_input')
decoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM)(decoder_input)
decoder_dropout = tf.keras.layers.Dropout(DROPOUT_RATE)(decoder_embedding)
decoder_lstm = tf.keras.layers.LSTM(UNITS, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_dropout, initial_state=encoder_states)
decoder_dense = tf.keras.layers.Dense(VOCAB_SIZE, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [64]:
model = tf.keras.Model([encoder_input, decoder_input], decoder_outputs)

In [66]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_input             │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_input             │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_6 (Embedding)   │ (None, None, 64)       │        128,000 │ encoder_input[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_7 (Embedding)   │ (None, None, 64)       │        128,000 │ decoder_input[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, None, 64)       │              0 │ embedding_6[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, None, 64)       │              0 │ embedding_7[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_6 (LSTM)             │ [(None, 128), (None,   │         98,816 │ dropout[0][0]          │
│                           │ 128), (None, 128)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_7 (LSTM)             │ [(None, None, 128),    │         98,816 │ dropout_1[0][0],       │
│                           │ (None, 128), (None,    │                │ lstm_6[0][1],          │
│                           │ 128)]                  │                │ lstm_6[0][2]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, None, 2000)     │        258,000 │ lstm_7[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 711,632 (2.71 MB)

 Trainable params: 711,632 (2.71 MB)

 Non-trainable params: 0 (0.00 B)

In [67]:
def prepare_data(contexts, targets):
    context_sequences = context_vectorizer(contexts)
    target_sequences = target_vectorizer(targets)

    decoder_input_seq = target_sequences[:, :-1]
    decoder_target_seq = target_sequences[:, 1:]

    return (context_sequences, decoder_input_seq), decoder_target_seq

In [68]:
train_data = prepare_data(context_raw[is_train], target_raw[is_train])
val_data = prepare_data(context_raw[~is_train], target_raw[~is_train])

In [81]:
def augment_text(sequence):
    words = sequence.split()
    np.random.shuffle(words)
    return ' '.join(words)

In [82]:
augmented_contexts = [augment_text(text) for text in context_raw[is_train]]
augmented_targets = [augment_text(text) for text in target_raw[is_train]]

In [83]:
augmented_contexts = np.array(context_raw[is_train].tolist() + augmented_contexts)
augmented_targets = np.array(target_raw[is_train].tolist() + augmented_targets)

In [84]:
train_data_augmented = prepare_data(augmented_contexts, augmented_targets)

In [85]:
EPOCHS = 20

history = model.fit(
    x=train_data[0],
    y=train_data[1],
    epochs=EPOCHS,
    validation_data=(val_data[0], val_data[1])
)

Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 332ms/step - accuracy: 1.0000 - loss: 0.0013 - val_accuracy: 1.0000 - val_loss: 0.0013
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 358ms/step - accuracy: 1.0000 - loss: 0.0013 - val_accuracy: 1.0000 - val_loss: 0.0012
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 262ms/step - accuracy: 1.0000 - loss: 0.0012 - val_accuracy: 1.0000 - val_loss: 0.0011
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 316ms/step - accuracy: 1.0000 - loss: 0.0011 - val_accuracy: 1.0000 - val_loss: 0.0011
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 346ms/step - accuracy: 1.0000 - loss: 0.0011 - val_accuracy: 1.0000 - val_loss: 0.0010
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 261ms/step - accuracy: 1.0000 - loss: 9.9774e-04 - val_accuracy: 1.0000 - val_loss: 9.5573e-04
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 10s 265ms/step - accuracy: 1.0000 - loss: 9.4564e-04 - val_accuracy: 1.0000 - val_loss: 9.0645e-04
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 10s 262ms/step - accuracy: 1.0000 - loss: 8.97